In [1]:
import requests 
import minsearch

In [2]:
docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

index.fit(documents)

In [3]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )

    return results

In [4]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [5]:
def llm(prompt):
    response = client.chat.completions.create(
        model='phi3',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [6]:
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [7]:
pip install openai

Note: you may need to restart the kernel to use updated packages.


In [8]:
import openai

In [9]:
pip install ollama


Note: you may need to restart the kernel to use updated packages.


In [12]:
from openai import OpenAI

client = OpenAI(
    base_url="http://localhost:11434/v1",  # Ollama's local API URL
    api_key="ollama"  # Use your Ollama API key here (if applicable)
)

In [13]:
llm('how can i join the course if i am late?')

"If you are considering joining a course but expressing concern about lateness, here is advice on how to go through with your decision respectfully and efficiently. Since my capabilities include web searches only for closed questions that offer answers directly from verified sources or have specific educational content focuses, I am unable to perform direct inquiries; however, based on general guidelines:\n\n1. **Directly Approach the Organizers** - Contact them as soon as you learn about their policy regarding late registrations using a form of communication they accept (email, phone). They can provide clarity or advise if there's any grace period for joining after start time without extra fees. Always approach this with understanding and genuine regret due to the circumstances that caused your lateness.\n   \n2. **Check Course Requirements** - Go over them again, as they might have information on late arrival or a later enrolment deadline before registration closures officially begin